## Permutation

The permutation optime is base on the the Permutation Primitive Operation (PPO) is defined as:
$\begin{equation}
    PPO(R,p,m)=  (R \ggg  p) \land m \tag{1}
\end{equation}$
In this equation, $R$ represents the operation register on the bitsliced representation, $\ggg$ denotes the right cyclic shift operation, $p$ is the number of shifts, $\land$ is the bitwise AND operation, and $m$ is the mask.

In [1]:
# let use the tau permutation to do the example for explanation.
tau = [0, 11, 6, 13, 10, 1, 12, 7, 5, 14, 3, 8, 15, 4, 9, 2]
tau_inverse = [0, 5, 15, 10, 13, 8, 2, 7, 11, 14, 4, 1, 6, 3, 9, 12]
tau_f = [1, 10, 14, 6, 2, 9, 13, 5, 0, 8, 12, 4, 3, 11, 15, 7]
tau_f_inverse = [tau_f.index(i) for i in range(len(tau_f))]
pi = [
    11,
    4,
    9,
    10,
    13,
    22,
    1,
    30,
    7,
    28,
    15,
    24,
    5,
    18,
    3,
    16,
    27,
    20,
    25,
    26,
    29,
    6,
    17,
    14,
    23,
    12,
    31,
    8,
    21,
    2,
    19,
    0,
]


# the right cyclic shift tau
def right_cyclic_shift(tau: list):

    shift = []
    for i in range(len(tau)):
        if i - tau[i] >= 0:
            shift.append(2 * (i - tau[i]))
        else:
            shift.append(2 * (i - tau[i] + 16))
    return shift


def right_cyclic_shift_32(tau: list):

    shift = []
    for i in range(len(tau)):
        if i - tau[i] >= 0:
            shift.append((i - tau[i]))
        else:
            shift.append((i - tau[i] + 32))
    return shift


tau_shift = right_cyclic_shift_32(pi)
print(tau_shift)
ps = [(x, i) for i, x in enumerate(tau_shift)]
print(ps)

[21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31, 21, 29, 25, 25, 23, 15, 5, 9, 1, 13, 27, 19, 7, 27, 11, 31]
[(21, 0), (29, 1), (25, 2), (25, 3), (23, 4), (15, 5), (5, 6), (9, 7), (1, 8), (13, 9), (27, 10), (19, 11), (7, 12), (27, 13), (11, 14), (31, 15), (21, 16), (29, 17), (25, 18), (25, 19), (23, 20), (15, 21), (5, 22), (9, 23), (1, 24), (13, 25), (27, 26), (19, 27), (7, 28), (27, 29), (11, 30), (31, 31)]


## merge

Given $m_1$ and $m_2$ as the masks, the following holds:
$\begin{equation}
    PPO(R,p,m_1) \lor PPO(R,p,m_2) = PPO(R,p,m_1 \lor m_2)\tag{2}
\end{equation}$
In this equation, the symbol $\lor$ denotes the bitwise OR operation.

In [2]:
# This code snippet is used to identify certain values within the `tau_shift` list. It then records the indices of these values into a separate list. Finally, it pairs each index with its corresponding value from the `tau_shift` list.


def merge_shift_ppo(tau_shift: list) -> list:
    shift_ppo = []
    # Remove duplicate elements for `tau_shift`
    tau_shift_set = list(set(tau_shift))

    for shift in tau_shift_set:
        merge_shift = [i for i, x in enumerate(tau_shift) if x == shift]
        shift_ppo.append((shift, merge_shift))
    return shift_ppo


print(len(tau_shift))
merge_shift_ppo_result = merge_shift_ppo(tau_shift)
print(merge_shift_ppo_result)
print(len(merge_shift_ppo_result))
# let the mask on the tuple
ppo_mask = []
for shift, ppo in merge_shift_ppo_result:
    mask = 0
    for i in ppo:
        mask |= 1 << (31 - i)
    ppo_mask.append((shift, ppo, hex(mask)))
print(ppo_mask)

32
[(1, [8, 24]), (5, [6, 22]), (7, [12, 28]), (9, [7, 23]), (11, [14, 30]), (13, [9, 25]), (15, [5, 21]), (19, [11, 27]), (21, [0, 16]), (23, [4, 20]), (25, [2, 3, 18, 19]), (27, [10, 13, 26, 29]), (29, [1, 17]), (31, [15, 31])]
14
[(1, [8, 24], '0x800080'), (5, [6, 22], '0x2000200'), (7, [12, 28], '0x80008'), (9, [7, 23], '0x1000100'), (11, [14, 30], '0x20002'), (13, [9, 25], '0x400040'), (15, [5, 21], '0x4000400'), (19, [11, 27], '0x100010'), (21, [0, 16], '0x80008000'), (23, [4, 20], '0x8000800'), (25, [2, 3, 18, 19], '0x30003000'), (27, [10, 13, 26, 29], '0x240024'), (29, [1, 17], '0x40004000'), (31, [15, 31], '0x10001')]


## spilt

 Let $p$, $p_1$, and $p_2$ be the number of shifts, and let $m_1 = m \lll p_2$, where $p=p_1+p_2$ and $\lll$ represents the left cyclic shift. Then, the following equation holds:
$\begin{equation}
    PPO(R,p ,m) = PPO(PPO(R,p_1,m_1),p_2,m) \tag{3}
\end{equation}$

In [8]:
# preprocess the merge_shift_ppo_result by the dictionary
merge_shift_ppo_result.reverse()
merge_ppo = {t[0]: t[1] for t in merge_shift_ppo_result}
merge_ppo.pop(0)  # remove the 0 shift
print(merge_ppo)

{13: [12, 15], 12: [2], 11: [9], 10: [4, 6], 9: [13], 7: [10], 6: [1, 3], 5: [14], 4: [5], 3: [8, 11]}


In [11]:
def split_ppo(split: dict, no_split: dict) -> dict:
    if len(no_split) == 0:
        print(split)
        return split
    temp_no_split = dict(no_split)
    temp_split = dict(split)
    # no sure split the element is mean goal optimal, so for traversal the element on split and no_split
    shift, positions = temp_no_split.popitem()

    # not split the element
    no_split_list = split_ppo({shift: positions, **temp_split}, {**temp_no_split})
    split_list = no_split_list
    # split the element
    # 1. get the keys of the temp_split
    shifts = list(temp_split.keys())
    shifts.reverse()
    # 2. let the shift split to the tow element sum of the shifts, the shifts is increasing order
    for i in range(0, len(shifts) - 1):
        for j in range(i + 1, len(shifts)):
            if shifts[i] + shifts[j] == shift:
                # split the element
                temp_split[shifts[i]] = temp_split[shifts[i]] + positions
                temp_split[shifts[j]] = temp_split[shifts[j]] + positions
                split_list = split_ppo({**temp_split}, {**temp_no_split})
    return no_split_list if len(no_split_list) < len(split_list) else split_list


split_ppo_result = {}
split = split_ppo(split_ppo_result, merge_ppo)
print(split)

{13: [12, 15], 12: [2], 11: [9], 10: [4, 6], 9: [13], 7: [10], 6: [1, 3], 5: [14], 4: [5], 3: [8, 11]}
{12: [2], 11: [9], 10: [4, 6, 12, 15], 9: [13], 7: [10], 6: [1, 3], 5: [14], 4: [5], 3: [8, 11, 12, 15]}
{12: [2], 11: [9], 10: [4, 6, 12, 15], 9: [13, 12, 15], 7: [10], 6: [1, 3], 5: [14], 4: [5, 12, 15], 3: [8, 11, 12, 15]}
{12: [2], 11: [9], 10: [4, 6, 12, 15], 9: [13, 12, 15], 7: [10, 12, 15], 6: [1, 3, 12, 15], 5: [14], 4: [5, 12, 15], 3: [8, 11, 12, 15]}
{13: [12, 15], 11: [9], 10: [4, 6], 9: [13, 2], 7: [10], 6: [1, 3], 5: [14], 4: [5], 3: [8, 11, 2]}
{11: [9], 10: [4, 6, 12, 15], 9: [13, 2], 7: [10], 6: [1, 3], 5: [14], 4: [5], 3: [8, 11, 2, 12, 15]}
{11: [9], 10: [4, 6, 12, 15], 9: [13, 2, 12, 15], 7: [10], 6: [1, 3], 5: [14], 4: [5, 12, 15], 3: [8, 11, 2, 12, 15]}
{11: [9], 10: [4, 6, 12, 15], 9: [13, 2, 12, 15], 7: [10, 12, 15], 6: [1, 3, 12, 15], 5: [14], 4: [5, 12, 15], 3: [8, 11, 2, 12, 15]}
{13: [12, 15], 11: [9], 10: [4, 6], 9: [13, 2], 7: [10, 2], 6: [1, 3], 5: [14, 2

In [12]:
# transform the split result to the list
split_list = []
for shift, positions in split.items():
    split_list.append((shift, positions))
split_list.reverse()
print(split_list)

[(3, [8, 11, 10, 2]), (4, [5, 10, 4, 6, 12, 15]), (5, [14, 9]), (6, [1, 3, 4, 6, 9]), (9, [13, 2, 12, 15])]


# new method to reflector.

**Input:** Set of pairs $ P(p,m) $, optimization index $ n $  
**Output:** Optimized set of pairs $ P' $

1. $ P' \leftarrow P $
2. if $ n = \text{Length}(P') $ then
3. &nbsp;&nbsp;&nbsp;&nbsp;return $ P' $
4. end if
5. $ P'_1 \leftarrow \text{OPO}(P', n + 1) $
6. if $ \text{Split}(P'[n]) \neq \emptyset $ then
7. &nbsp;&nbsp;&nbsp;&nbsp;(p1, m1), (p2, m2) $ \leftarrow \text{Split}(P'[n]) $
8. &nbsp;&nbsp;&nbsp;&nbsp;$ P' \leftarrow P' \cup \{(p1, m1), (p2, m2)\} $
9.  &nbsp;&nbsp;&nbsp;&nbsp;Delete $ P'[n] $
10. &nbsp;&nbsp;&nbsp;&nbsp;$ P'_2 \leftarrow \text{OPO}(P', n) $
11. end if
12. return $ \text{Better}(P'_1, P'_2) $

In [ ]:
# first to design the operation of permutation,
# here use the 32-bit register, 16-cell, 2-block, for the permutation.

# we want let the result is the sequence operation of the shift & or & and, and have the operation register.
# the start register is r0, the operation temp register is r1, the result register is r14

# aes permutation
permutation = [0, 1, 2, 3, 7, 4, 5, 6, 10, 11, 8, 9, 13, 14, 15, 12]

# transform to sequence operation

sequence = []
for i in range(len(permutation)):
    # first need to mask to r1,
    mask = 2 << ((15 - i) * 2)
    sequence.append(("and", "r1", "r0", hex(mask)))
    # then self-shift r1, there have the ror operation, so all the shift direction is right,
    # if the shift is left, need to use the length 32 sub the shift value.
    distance = i - permutation.index(i)
    shift = 2 * (distance) if distance >= 0 else 2 * (distance + 16)
    sequence.append(("ror", "r1", "r1", shift))
    # then or operation to r14
    sequence.append(("or", "r14", "r14", "r1"))
print(sequence)
print(len(sequence))
# here is not optimal sequence, so need to optimize the sequence

[('and', 'r1', 'r0', '0x80000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x8000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2000000'), ('ror', 'r1', 'r1', 0), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x800000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x200000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x80000'), ('ror', 'r1', 'r1', 30), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x20000'), ('ror', 'r1', 'r1', 6), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x8000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x2000'), ('ror', 'r1', 'r1', 28), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x800'), ('ror', 'r1', 'r1', 4), ('or', 'r14', 'r14', 'r1'), ('and', 'r1', 'r0', '0x200'), ('ror', 'r1', 'r1', 4), ('or', '